In [7]:
# https://openmm.github.io/openmm-cookbook/latest/notebooks/cookbook/Merging%20Molecules.html

from openmm.app import *
import nglview as nv

pdb1 = PDBFile('./data/villin.pdb')
pdb2 = PDBFile('./data/ala_ala_ala.pdb')


In [5]:
print(pdb1.topology)

<Topology; 1 chains, 2798 residues, 8867 atoms, 6111 bonds>


In [6]:
print(pdb2.topology)

<Topology; 1 chains, 3 residues, 33 atoms, 32 bonds>


In [8]:
nv.show_structure_file('./data/villin.pdb')

NGLWidget()

In [9]:
nv.show_structure_file('./data/ala_ala_ala.pdb')

NGLWidget()

In [10]:

modeller = Modeller(pdb1.topology, pdb1.positions)
modeller.add(pdb2.topology, pdb2.positions)
#mergedTopology = modeller.topology
#mergedPositions = modeller.positions

#print(mergedTopology)
#print(mergedPositions)

In [11]:
modeller.topology

<Topology; 2 chains, 2801 residues, 8900 atoms, 6143 bonds>

In [13]:
with open('./results/merged.pdb', 'w') as f:
    PDBFile.writeFile(modeller.topology, modeller.positions, f)

In [38]:
#pdb3 = PDBFile('./results/merged.pdb')
merged_view = nv.show_structure_file('./results/merged.pdb')
#merged_view.representation = 'cartoon'
merged_view.add_representation('licorice', selection='not protein')
merged_view.add_representation('spacefill', selection='protein')
merged_view
#nv.show_structure_file('./results/merged.pdb')

NGLWidget()

## 4 Hydrogen atoms

### Adding Hydrogens to Nonstandard Molecules
* https://openmm.github.io/openmm-cookbook/latest/notebooks/cookbook/Adding%20Hydrogens%20to%20Nonstandard%20Molecules.html
* Modeller.addHydrogens() can be used to add missing hydrogens to standard molecules, like proteins and nucleic acids. But what if you need to simulate something it doesn’t know about, such as a drug molecule or a non-natural amino acid? It can also handle this case, but you need to give it some help.

* Start by creating an XML file describing the molecule or molecules. Here is an example of a file defining hydrogens for NLN, a modified ASN residue that is missing a hydrogen and can have a glycan bonded to it. (This example is taken from the glycam-hydrogens.xml file that is bundled with OpenMM. You would not actually need to define this particular residue yourself.)

In [41]:
''''# MyHydrogen.xml
<Residues>
  <Residue name="NLN">
    <H name="H" parent="N" terminal="-C"/>
    <H name="H1" parent="N" terminal="N"/>
    <H name="H2" parent="N" terminal="N"/>
    <H name="H3" parent="N" terminal="N"/>
    <H name="HA" parent="CA"/>
    <H name="HB2" parent="CB"/>
    <H name="HB3" parent="CB"/>
    <H name="HD21" parent="ND2"/>
  </Residue>
</Residues>
'''

'\'# MyHydrogen.xml\n<Residues>\n  <Residue name="NLN">\n    <H name="H" parent="N" terminal="-C"/>\n    <H name="H1" parent="N" terminal="N"/>\n    <H name="H2" parent="N" terminal="N"/>\n    <H name="H3" parent="N" terminal="N"/>\n    <H name="HA" parent="CA"/>\n    <H name="HB2" parent="CB"/>\n    <H name="HB3" parent="CB"/>\n    <H name="HD21" parent="ND2"/>\n  </Residue>\n</Residues>\n'

* There is one <Residue> tag for every residue you want to define.

* It contains one <H> tag for every hydrogen that can appear in the residue.

* 'parent' is the name of the heavy atom the hydrogen is bonded to.

* All atom and residue names must exactly match the names present in your Topology.

* For terminal argument:
    * The optional terminal attribute indicates hydrogens that might or might not be present, depending on the residue’s position in the chain. It should contain one or more of the characters “N”, “C”, and “-”. 
    * “N” indicates the hydrogen should be added to N-terminal residues. 
    * “C” indicates it should be added to C-terminal ones. 
    * “-” indicates it should be added to residues that are not at either end of the chain.

In [43]:
Modeller.loadHydrogenDefinitions('./data/MyHydrogen.xml')

In [44]:
modeller = Modeller(pdb1.topology, pdb1.positions)
modeller.add(pdb2.topology, pdb2.positions)
mergedTopology = modeller.topology
mergedPositions = modeller.positions

In [45]:
with open('./results/merged_H.pdb', 'w') as f:
    PDBFile.writeFile(mergedTopology, mergedPositions, f)

In [46]:

merged_H_view = nv.show_structure_file('./results/merged_H.pdb')
#merged_view.representation = 'cartoon'
merged_H_view.add_representation('licorice', selection='not protein')
merged_H_view.add_representation('spacefill', selection='protein')
merged_H_view
#nv.show_structure_file('./results/merged.pdb')

NGLWidget()